# Part 1 - System setup and visualisation

This section of the tutorial will cover setting up an ATM-compatible system within BioSimSpace, and visualising the system, as well as the choices made during setup.

First, import BioSimSpace

In [ ]:
import BioSimSpace as BSS

In order speed things up, this tutorial will use pre-parameterised proteins and ligands. It is however possible to parameterise molecules from a variety of file formats yourself using functionality in `BioSimSpace.IO` and `BioSimSpace.Parameters`.

In [ ]:
# Use BioSimSpace IO to read inputs, specifically the pre-parameterised protein and pair of ligands.
url = BSS.tutorialUrl()
protein = BSS.IO.readMolecules([f"{url}/tyk2.prm7", f"{url}/tyk2.rst7"])[0]
ligand_bound = BSS.IO.readMolecules([f"{url}/ejm_31.prm7", f"{url}/ejm_31.rst7"])[0]
ligand_free = BSS.IO.readMolecules([f"{url}/ejm_43.prm7", f"{url}/ejm_43.rst7"])[0]

In [ ]:
# We also need to download the example output files
from get_tutorial import download

download("01")

The protein used in this case is tyrosine kynase 2, or TYK2, a common benchmark system for binding free energy calculations.

In [ ]:
view = BSS.Notebook.View(protein)
view.system()

Unlike in other RBFE methods, a system used in ATM calculation includes both ligands simultaneously, with one ligand bound to the protein and the other free in the solvent. 

We will use BioSimSpace to create an ATM compatible system from the inputs loaded above, choosing which ligand is to be bound and which is to be free.

One decision that needs to be made prior to creating an ATM compatible system is the choice of "rigid core atoms" within each ligand. These atoms are used to define a series of aligning forces that will be used to maintain the relative orientation of the two ligands during minimisation, equilibration and production simulations. 

Maintaining the orientation of the ligands is important to the stability of ATM simulations as, in the second half of production (i.e. windows with a lambda value greater than 0.5) the free and bound ligands will be swapped. If the free ligand is out of alignment during this swap it will cause direct overlaps with protein atoms, causing the simulations to immediately explode.

BioSimSpace provides a helper function that visualises the choice of these atoms

In [ ]:
BSS.FreeEnergy.ATMSetup.viewRigidCores(
    ligand_bound=ligand_bound,
    ligand_free=ligand_free,
    ligand_bound_rigid_core=[14, 11, 15],
    ligand_free_rigid_core=[14, 11, 15],
)

Making a good choice of rigid core atoms can be complex. A good starting point is to find a common core between your two ligands and choose three well-separated atoms within this common core. Before making your own choice of rigid core atoms it is highly recommended to read the guide on the Gallichio lab website [here](https://www.compmolbiophysbc.org/atom-openmm/atom-system-setup).

Now that a sensible choice of rigid core atoms has been made we can now create the system using the `BioSimSpace.FreeEnergy.ATMSetup` class. 

In [ ]:
atm_setup = BSS.FreeEnergy.ATMSetup(
    receptor=protein, ligand_bound=ligand_bound, ligand_free=ligand_free
)
system, atm_data = atm_setup.prepare(
    ligand_bound_rigid_core=[14, 11, 15], ligand_free_rigid_core=[14, 11, 15]
)

System preparation returns a pair of objects, the first is a system containing the protein and two ligands, the second is a dictionary containing information on the setup of the system that will be used to ensure that consistent settings are used throughout all simulations. 

By default, BioSimSpace attempts to find a best-fit vector along which to translate the free ligand relative to the bound, and then translates the free ligand along this vector by a distance of 20 Angstroms. The translation vector can also be set manually by passing a displacement vector to `atm_setup.prepare`.

Now lets visualise the system to make sure that the free ligand is far enough away from the protein.

In [ ]:
view = BSS.Notebook.View(system)
view.system()

Although it might appear that the free ligand is too close to the protein, we only really need 3 layers of water (roughly 10 Angrstroms) between the two in order to have good separation, and so the separation in our system should be enough. Also, the smaller the separation the smaller the water box we will need, and the faster our simulations will run.

All that remains now is to solvate our protein-ligand-ligand system.

In [ ]:
solvated = BSS.Solvent.tip3p(molecule=system, box=3 * [7 * BSS.Units.Length.nanometer])

# Minimisation and Equilibration

ATM simulations require a significant number of restraints, as well as the gradual introduction of the ATMForce itself. As such standard minimisation and equilibration protocols are not appropriate, and a series of custom protocols is needed. 

In this section we will cover a full minimisation and equilibration protocol for the ATM system we created above. 

The simulations we will run in this section are significantly cut down, real production simulations should be minimised and equilibrated much more thoroughly than we will here, with at least 10000 minimisation steps and 100ps of runtime for each equilibration. 

ATM simulations are best run with a series of restraints:
- __core alignment__; these are the the rigid core restraints that are applied to the atoms we found earlier.
- __positional restraints__; these are flat-bottom restraints used to keep specific atoms in place, these are generally applied to the alpha carbons of the protein.
- __centre of mass distance restraints__; these are used to maintain the relative positions of the protein and ligands and are applied to the centre off mass of the atoms listed in the `data` dictionary (these were found automatically when we prepared the system, but they can be set manually if the ones found are not appropriate).

Now let's find the alpha carbons that we will be restraining using BioSimSpace search functionality.

In [ ]:
ca = [atom.index() for atom in solvated.search("atomname CA")]

Now we can create a minimisation protocol using the forces and `atm_data` dictionary we created when setting up the system.

In [ ]:
minimisation = BSS.Protocol.ATMMinimisation(
    steps=200,
    data=atm_data,
    core_alignment=True,
    restraint=ca,
    com_distance_restraint=True,
)

This protocol can then be run using the BioSimSpace OpenMM process. This minimisation may take a few minutes.

In [ ]:
minimisation_process = BSS.Process.OpenMM(solvated, minimisation)
minimisation_process.start()
minimisation_process.wait()
minimised = minimisation_process.getSystem(block=True)

We will now run our first equilibration, including all of the forces we used in the minimisation. 

In [ ]:
equilibration = BSS.Protocol.ATMEquilibration(
    data=atm_data,
    core_alignment=True,
    restraint=ca,
    com_distance_restraint=True,
    runtime="1ps",
)
equilibrate_process = BSS.Process.OpenMM(minimised, equilibration)
equilibrate_process.start()
equilibrate_process.wait()
equilibrated = equilibrate_process.getSystem(block=True)

The system has now been minimised and equilibrated as a vanilla system, that is without the ATMForce present. 

We now need to introduce the ATMForce to the system; this introduction needs to be gradual, otherwise our simulations will crash.

This gradual introduction can be done by annealing the system to a lambda value of 0.5. Starting from lambda=0, the annealing protocol simulates a series of windows in which the value of lambda is gradually increased. In this case we will use 10 annealing cycles with a total runtime of 1ps, meaning that the value of lambda is increased by a value of 0.05 every 0.1ps.



In [ ]:
annealing = BSS.Protocol.ATMAnnealing(
    data=atm_data,
    core_alignment=True,
    restraint=ca,
    com_distance_restraint=True,
    runtime="1ps",
    anneal_numcycles=10,
)
annealing_process = BSS.Process.OpenMM(equilibrated, annealing)
annealing_process.start()
annealing_process.wait()
annealed = annealing_process.getSystem(block=True)

Finally, we need to perform a post-annealing equilibration. This equilibration step is performed at a lambda value of 0.5, and is designed to introduce the free ligand to the protein-ligand complex without actually performing a swap. This step dramatically increases the stability of production simulations in which `ligand_free' is in the binding site (i.e. those with a lambda value greater than 0.5) by decreasing the probability of atom overlaps occurring when the ligand is placed in the binding site.

When running full production simulations this is the step that ytou should look to first if you experience instability. It is usually advisable to run this post-annealing equilibration for at least 500ps to give the system enough time to settle.

In [ ]:
post_anneal_equilibration = BSS.Protocol.ATMEquilibration(
    data=atm_data,
    core_alignment=True,
    restraint=ca,
    com_distance_restraint=True,
    use_atm_force=True,
    lambda1=0.5,
    lambda2=0.5,
    runtime="1ps",
)
post_anneal_equilibration_process = BSS.Process.OpenMM(
    annealed, post_anneal_equilibration
)
post_anneal_equilibration_process.start()
post_anneal_equilibration_process.wait()
min_eq_final = post_anneal_equilibration_process.getSystem(block=True)

Before proceeding, lets make sure our minimised and equilibrated system looks sensible

In [ ]:
view = BSS.Notebook.View(min_eq_final)
view.system()

The example outputs that we downloaded earlier contain a version of this system that has been properly minimised and equilibrated. Let's visualise it and compare it to the system we've prepared ourselves. Provided that these two systems look similar, and most importantly that the two ligands are still in alignment, we should now be ready for production.

In [ ]:
check = BSS.Stream.load("tyk2_atm/ready_for_production.bss")
view1 = BSS.Notebook.View(check)
view1.system()

# Production and Analysis

Now that the system has been properly minimised and equilibrated it is ready for production. 

Before running production simulations we need to decide how many lambda windows to run. Keep in mind that when performing an ATM calculation we are running the equivalent of all 4 legs (bound and free, forward and reverse) of a standard RBFE calculation simultaneously, so we should expect to have to run a larger number of windows.

A good middle ground for most systems is 22 windows, and this is the default chosen by BioSimSpace, but some systems might need more sampling.

For the TYK2 simulations we are running in this tutorial we will need to change some of the soft-core settings, as the BioSimSpace defaults are not appropriate.

In [ ]:
alpha = 22 * [0.1]
uh = 22 * [110.0]
output_directory = "tyk2_atm"
production_atm = BSS.Protocol.ATMProduction(
    data=atm_data,
    core_alignment=True,
    restraint=ca,
    com_distance_restraint=True,
    runtime="1ns",
    num_lambda=22,
    alpha=alpha,
    uh=uh,
)

We won't actually run any production simulations here, as they would take far too long, but if we did want to run production we could do so with this command: 

In [ ]:
# production_process = BSS.FreeEnergy.ATM(
#  system=min_eq_final,
#  protocol=production_atm,
#  work_dir=output_directory,
# )

Once production simulations are complete `output_directory` will be populated by a series of folders, one for each lambda window. Within each of these folders should be a file called `openmm.csv` which contains the information we need to calculate the $\Delta \Delta G$ value for our system. In this case we will be analysing the pre-prepared outputs that we downloaded earlier.

In-built BioSimSpace analysis will give us a $\Delta \Delta G$ value in kcal/mol, along with the error, also in kcal/mol.

In [ ]:
DDG = BSS.FreeEnergy.ATM.analyse(output_directory)
DDG